# Read in results

In [1]:
from sklearn.metrics import roc_auc_score
import sys
sys.path.append('text_nn')
import pandas as pd
import rationale_net
from rationale_net.datasets.new_york_times_dataset import NYTimesDataset
import pickle
results = pickle.load(open("exploratory_analysis/logs/results__model-form_cnn__num-layers_1__word-cutoff_110__epochs_18", 'rb'))

In [2]:
pd.Series(results)

train                                                              True
test                                                               True
cuda                                                               True
num_gpus                                                              1
debug_mode                                                        False
class_balance                                                     False
objective                                                 cross_entropy
aspect                                                          overall
init_lr                                                          0.0001
epochs                                                               18
batch_size                                                           64
patience                                                              5
tuning_metric                                                      loss
save_dir                                                       s

In [3]:
test_stats = pd.Series(results['test_stats'])

True ratio

In [4]:
pd.Series(results['test_stats']['golds']).value_counts()

0    246089
1     11379
dtype: int64

Predicted ratio

In [5]:
pd.Series(results['test_stats']['preds']).value_counts()

0    176288
1     81180
dtype: int64

In [6]:
y_proba = list(map(lambda x: x[1] if x[0] == 1 else 1-x[1], zip(test_stats['preds'], test_stats['probas'])))
y_true = results['test_stats']['golds']

ROC

In [8]:
roc_auc_score(y_true, y_proba)

0.6537010522091742

# Rationals

In [9]:
rationales = results['test_stats']['rationales']

In [10]:
rationales[4]

'_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ cazale _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _'

In [11]:
rationale_words = pd.Series(list(map(lambda r: ' '.join(r.replace('_', '').split()), rationales)))

In [12]:
rationale_words.str.split(' ').str.len().value_counts()

1     232062
2      20312
3       3847
4        944
5        210
6         69
7         17
8          6
10         1
dtype: int64

In [13]:
rational_df = pd.concat([
    rationale_words.to_frame('word_list'),
    pd.Series(y_true).to_frame('y_true'),
    pd.Series(results['test_stats']['preds']).to_frame('y_pred')
], axis=1)

Overall most popular rationales

In [14]:
rational_df['word_list'].value_counts().head(10)

nyt          6429
-            3075
:            2336
bush         2268
             1669
tuesday      1447
monday       1410
officials    1306
sunday       1284
wednesday    1268
Name: word_list, dtype: int64

Popular two-word rationales

In [15]:
rational_df['word_list'].loc[lambda s: s.str.split().str.len()==2].value_counts().head(10)

mourns mourns          393
nyt nyt                276
kristof kristof        225
( nyt                  147
nj nj                  122
krugman krugman         72
mcgreevey mcgreevey     66
safire safire           56
swarns nyt              52
mourn mourn             49
Name: word_list, dtype: int64

Popular rationales for y_pred = 0 (article != front page)

In [16]:
rational_df.loc[lambda df: df['y_pred'] == 0]['word_list'].value_counts().head(10)

nyt          6429
-            3075
:            2336
             1669
tuesday      1447
monday       1410
sunday       1284
wednesday    1268
thursday     1140
editor        825
Name: word_list, dtype: int64

Popular rationals for y_pred = 1 (article = front page)

In [17]:
rational_df.loc[lambda df: df['y_pred'] == 1]['word_list'].value_counts().head(10)

bush         2268
officials    1293
largest       992
bloomberg     793
nation        613
pataki        511
announced     418
experts       315
lawmakers     258
sgt           241
Name: word_list, dtype: int64